# Práctica: Inteligencia Artificial en Python

El objetivo de esta práctica es que el alumno se familiarice con la Inteligencia Artificial en un caso de estudio práctico. Para lo cual, continuando la practica anterior, se ha seleccionado un problema clásico en el procesado de nubes de puntos: la segmentación semántica de una nube. Con el mismo tipo de datos que en la Práctica 2, nubes de puntos adquiridas con Velodyne64 y disponibles en SemanticKITTI.

Un problema de segmentación semántica consiste en clasificar cada elemento básico que componen los datos. En imagen, se trata de pixeles, en nubes de puntos, se trata de puntos. A veces, la segmentación semántica también se denomina clasificación pixel por pixel, o punto por punto.


# Objetivos

En esta práctica vamos a ver fragmentos de código que expliquen cómo se preparar datos para un algoritmo de IA, entrenarlo y usarlo para realizar predicciones. Las operaciones que se estudiarán a continuación son:

- Lectura y escritura de nubes de puntos como datos txt
- Preparación de datos para Inteligencia Artificial
- Extracción de características
- Entrenamiento de algoritmos
- Análisis de los resultados mediante métricas: matrices de confusión, precision, recall, f1-score y accuracy
- Visualización de la clasificación

# Tarea 1

Los algoritmos empleados en inteligencia artificial son bastante complejos. Aunque podemos programar un algoritmo con conocimientos simples sobre el tema, conseguir que alcance las tasas de acierto de algoritmos ya desarrollados por otros autores requiere un fuerte conocimeinto en matemáticas y computación, además de pontentes servidores donde realizar pruebas. Por suerte, la mayoría de algoritmos de IA están en código abierto y recopilados en librerías.

La primera tarea consiste en preparar el notebook para trabajar con él:
- Carga en el direcorio las carpetas *Nubes* e *img*, 
- Instala la librería *pyntcloud*
- Comprueba si la librería sklearn está instalada. **Nota**: sklearn aparece en el instalador como scikit-learn.

Puedes encontrar la ayuda de esta librería en:
- https://scikit-learn.org/stable/

Una vez instalada la librería, vamos a importar todas las funciones que necesitamos. Si alguna da error, comprueba que está instalada en el entorno.

In [2]:
import numpy as np
from pyntcloud import PyntCloud
import pandas as pd
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Lectura y escritura de nubes de puntos como datos txt

Los algoritmos basados en IA necesitan de una fase de entrenamiento antes de poder emplearlos para clasificar nuevos datos. Como dato de entrada vamos a emplear las nubes de puntos 000036 (train) y 000079 (test). Pero en este caso no vamos a cargarlas como pcd, si no como txt, puesto que este tipo de formato, junto con csv, es el más común en el que podemos encontrar otros tipos de datos para IA. 

In [3]:
# Lectura de datos
train_data = np.loadtxt("Nubes/000036.txt", delimiter=' ')
#Visualización
print(train_data)

[[-17.16173553  11.24152946  -2.31720781   3.        ]
 [  4.90871477   5.81539869  -1.88928902   3.        ]
 [ -1.45642543  -5.4461484   -1.49202049   3.        ]
 ...
 [ -3.27209139  -6.23964262   0.18170004   2.        ]
 [ -4.83830881  -6.34132957  -0.91975325   2.        ]
 [  2.9530251    3.6776309   -1.87011027   3.        ]]


La organización de los datos en IA se distribuye en matrices donde cada fila se corresponde con una muestra y cada columna con un atributo. Esto es muy similar a como se distribuyen las nubes de puntos. En los datos anteriores podemos ver que cada punto contiene 3 coordeandas y la cuarta columna se corresponde con la etiqueta siguiendo el siguiente código:
- 1: coche
- 2: edificio
- 3: suelo
- 4: vegetación

# Preparación de datos para Inteligencia Artificial

Pero estos datos no pueden emplearse directamente en un algoritmo de IA, deben dividirse en una matriz de atributos NxM, siendo N el número de muestras y M el número de atributos, y una matriz de etiquetas (Nx1).

Además, en nubes de puntos no se pueden emplear las coordenadas como atributos de entrenamiento para IA, puesto que la posición de los puntos en coordenadas absolutas no representa ni tiene relación con nuevos datos. En nubes de puntos, las coordenadas son empleadas para extraer nuevas características geométricas que sí son aptas para el clasificador.

Por lo tanto, antes de nada, vamos a dividir la matriz de entrada en dos matrices:
- Matriz de coordenadas (definida como un dataframe con título de columnas, necesario para su transformación a Pyntcloud object)
- Matriz de etiquetas (definida como numpy array)

In [4]:
# Extraer matriz de coordenadas
coord = pd.DataFrame(list(zip(train_data[:,0],train_data[:,1],train_data[:,2])))  

# Asignar título a columnas
coord.columns =['x', 'y', 'z']

#Visualizar
print(coord)

                x          y         z
0      -17.161736  11.241529 -2.317208
1        4.908715   5.815399 -1.889289
2       -1.456425  -5.446148 -1.492020
3       -1.972294  11.678061 -0.694271
4       -7.067206  -6.265741 -0.708958
...           ...        ...       ...
112686   7.683413  -6.757722 -0.500771
112687   3.666602  -6.263341 -0.079479
112688  -3.272091  -6.239643  0.181700
112689  -4.838309  -6.341330 -0.919753
112690   2.953025   3.677631 -1.870110

[112691 rows x 3 columns]


In [0]:
# Extraer matriz de etiquetas
train_labels = train_data[:,3]

#Visualizar
print(train_labels)

# Extracción de características

Como se menciona anteriormente, las coordenadas no son útiles para el entrenamiento de un algoritmo basado en IA, y por lo tanto tenemos que extraer características geométricas. Aunque podemos calcularlas empleada directamente la matriz, es más fácil convertir la matriz a un objeto cloud en la librería pyntcloud y emplear sus funciones para calcularlas, como en la Práctica 2. 

Para una conversión correcta, cloud no admite un numpyarray, si no que tiene que emplearse un dataframe con los títulos "x", "y", y "z", calculados en el paso anterior.

Las características empleadas para el entrenamiento serán:
- Normales
- Curvatura
- Omnivariaza
- Linealidad
- Planaridad
- Dispersion

El uso de estas características está ampliamente extendido en nubes de puntos, su cálculo se encuentra explicado en el siguiente trabajo científico:
- Weinmann, M., Jutzi, B., & Mallet, C. (2014). Semantic 3D scene interpretation: A framework combining optimal neighborhood size selection with relevant features. ISPRS Annals of the Photogrammetry, Remote Sensing and Spatial Information Sciences, 2(3), 181.

In [5]:
#Conversion
cloud = PyntCloud(coord)

#Visualización
print(cloud)

PyntCloud
112691 points with 0 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -0.3115564670011339, 0.6556871394265795, -1.1372909533108966
Other attributes:



# Tarea 2

Calcula las normales para 25 vecinos.

In [6]:
# Cálculo de 25 vecinos
k_neighbors_25 = COMPLETAR

# Cálculo de normales
COMPLETAR

#Visualización
cloud.points

,x,y,z,nx(26),ny(26),nz(26)
0,-17.161736,11.241529,-2.317208,0.409398,-0.516128,0.752333
1,4.908715,5.815399,-1.889289,-0.036945,-0.011548,0.999251
2,-1.456425,-5.446148,-1.492020,-0.001438,0.117189,0.993109
3,-1.972294,11.678061,-0.694271,0.029662,-0.993468,0.110186
4,-7.067206,-6.265741,-0.708958,0.002528,0.999790,0.020353
...,...,...,...,...,...,...
112686,7.683413,-6.757722,-0.500771,0.354603,-0.929409,0.102250
112687,3.666602,-6.263341,-0.079479,-0.031530,0.998498,0.044809
112688,-3.272091,-6.239643,0.181700,-0.053892,0.978357,0.199785
112689,-4.838309,-6.341330,-0.919753,-0.014882,-0.994041,0.107983


A continuación, calcularemos las características basadas en autovalores

In [7]:
# Cálulo de autovalores
eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)

# Cálculo de características
cloud.add_scalar_field("curvature", ev=eigenvalues)
cloud.add_scalar_field("omnivariance", ev=eigenvalues)
cloud.add_scalar_field("linearity", ev=eigenvalues)
cloud.add_scalar_field("planarity", ev=eigenvalues)
cloud.add_scalar_field("sphericity", ev=eigenvalues)

# Visualización de los datos
cloud.points

,x,y,z,nx(26),ny(26),nz(26),e1(26),e2(26),e3(26),curvature(26),omnivariance(26),linearity(26),planarity(26),sphericity(26)
0,-17.161736,11.241529,-2.317208,0.409398,-0.516128,0.752333,0.068245,0.021763,0.011965,0.117333,0.026095,0.681104,0.143574,0.175322
1,4.908715,5.815399,-1.889289,-0.036945,-0.011548,0.999251,0.025810,0.002773,0.000005,0.000187,0.000726,0.892562,0.107231,0.000207
2,-1.456425,-5.446148,-1.492020,-0.001438,0.117189,0.993109,0.006031,0.003793,0.000003,0.000305,0.000409,0.371051,0.628452,0.000497
3,-1.972294,11.678061,-0.694271,0.029662,-0.993468,0.110186,0.005337,0.004681,0.001161,0.103868,0.003073,0.122956,0.659481,0.217563
4,-7.067206,-6.265741,-0.708958,0.002528,0.999790,0.020353,0.005885,0.005260,0.000068,0.006097,0.001284,0.106104,0.882278,0.011618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112686,7.683413,-6.757722,-0.500771,0.354603,-0.929409,0.102250,0.005997,0.003521,0.000738,0.071995,0.002498,0.412937,0.463938,0.123125
112687,3.666602,-6.263341,-0.079479,-0.031530,0.998498,0.044809,0.002252,0.001927,0.000296,0.066193,0.001087,0.144553,0.723923,0.131524
112688,-3.272091,-6.239643,0.181700,-0.053892,0.978357,0.199785,0.003350,0.002719,0.000325,0.050888,0.001437,0.188195,0.714662,0.097143
112689,-4.838309,-6.341330,-0.919753,-0.014882,-0.994041,0.107983,0.004117,0.002803,0.000779,0.101132,0.002079,0.319216,0.491678,0.189106


Para ser usadas en el algoritmo, las características deben ser devueltas al formato numpy array. Además, no todas las características de "points" son útiles, seleccionaremos las normales y las calculadas de los autovalores, pero no las coordenadas ni los autovalores en sí mismos.

In [8]:
# Selección de características
train_features = cloud.points[['nx(26)','ny(26)','nz(26)','curvature(26)','omnivariance(26)','linearity(26)','planarity(26)','sphericity(26)',]].to_numpy()
print(train_features)

[[ 4.09398236e-01 -5.16128039e-01  7.52332993e-01 ...  6.81103877e-01
   1.43574314e-01  1.75321809e-01]
 [-3.69452005e-02 -1.15478178e-02  9.99250569e-01 ...  8.92562219e-01
   1.07230621e-01  2.07160128e-04]
 [-1.43836749e-03  1.17188632e-01  9.93108632e-01 ...  3.71051005e-01
   6.28451662e-01  4.97333005e-04]
 ...
 [-5.38916500e-02  9.78356578e-01  1.99785126e-01 ...  1.88194981e-01
   7.14662256e-01  9.71427627e-02]
 [-1.48821685e-02 -9.94041318e-01  1.07983233e-01 ...  3.19215819e-01
   4.91678057e-01  1.89106123e-01]
 [ 6.81818376e-03  6.05997363e-02  9.98138860e-01 ...  2.59717123e-01
   7.39134624e-01  1.14825300e-03]]


# Funciones

Antes de proseguir, vamos a definir unas funciones

Hemos visto paso por paso como leer los datos de entrada, dividirlos y extraer sus características. Estas características son las empleadas en el entrenamiento, pero también son necesarias para la clasificación futura, por lo tanto, es necesario para cada muestra extraer estas características y generar una matriz de datos cuyos atributos se correspondan con el orden de entrenamiento. Para no repetir código cada vez que carguemos una matriz de datos, definiremos dos funciones. La primera separará los datos. La segunda, extraerá las características.

In [9]:
# Esta función servirá para separar los datos de entrada en coordenadas y etiquetas
def separar_input(input_matrix):
    coord = pd.DataFrame(list(zip(input_matrix[:,0],input_matrix[:,1],input_matrix[:,2])))  
    coord.columns =['x', 'y', 'z']
    labels = input_matrix[:,3]
    return coord, labels

# Tarea 3

Genera una función para la extracción de características que integre los pasos anteriores

In [10]:
def extraer_features(coord):
    # Crear nube
    cloud = COMPLETAR
    # Calcular vecinos
    k_neighbors_25 = COMPLETAR
    # Calcular y añadir normales
    cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)
    # Calcular y añadir eigenvalues
    eigenvalues = COMPLETAR
    # Calcular y añadir otras caracteristicas geometricas
    cloud.add_scalar_field("curvature", ev=eigenvalues)
    COMPLETAR
    COMPLETAR
    COMPLETAR
    COMPLETAR
    # Transformar dataframe de puntos a nparray de features (por orden)
    features = COMPLETAR
    return features

Si has sido capaz de copiar y pegar correctamente el código anterior para generar la función, el nuevo código para dividir y extraer características queda resumido en dos lineas:

In [11]:
# Separar datos de entrada
train_coord,train_labels = separar_input(train_data)

# Extraer características
train_features = extraer_features(train_coord)

# Entrenamiento de algoritmos

En este paso llegamos al núcleo del algoritmo. Una vez todos los datos están preparados procedemos al entrenamiento. En esta práctica vamos a entrenar y usar los dos algoritmos más empleados en la actualidad: 
- Support Vector Machine. Dado un conjunto de puntos, en el que cada uno de ellos pertenece a una de dos posibles categorías, un algoritmo basado en SVM construye un modelo capaz de predecir si un punto nuevo (cuya categoría desconocemos) pertenece a una categoría o a la otra.

<center> <img src="img/svm.png"></center>
<center>Fuente: https://en.wikipedia.org/wiki/Support-vector_machine</center>

- Random Forest. Es una combinación de árboles predictores tal que cada árbol depende de los valores de un vector aleatorio probado independientemente y con la misma distribución para cada uno de estos. El resultado de la calsificación es la clase predicha mayoritariamente por todos los árboles.

<center> <img src="img/rf.png"></center>
<center>Fuente: https://es.wikipedia.org/wiki/Random_forest</center>

Una vez realizado todo el trabajo, solo hay que darle los datos al algoritmo para que entrene.

In [12]:
# Definir clasificador SVM
clf_svm = svm.SVC()

#Entrenar (esto puede tardar varios minutos)
clf_svm.fit(train_features, train_labels)

SVC()

In [13]:
# Definir clasificador RF
clf_rf = RandomForestClassifier()

#Entrenar (esto debería ir más rápido que el anterior)
clf_rf.fit(train_features, train_labels)

RandomForestClassifier()

# Análisis de los resultados mediante métricas

Las métricas nos permiten saber cuán bueno es nuestro algoritmo entrenado respecto a la realidad. Si las métricas se aplican sobre los datos de entrenamiento sabremos si nuestro algoritmo ha sido capaz de aprender de los datos proporcionados. Pero para obtener resultados fiables y contrastables sobre el buen funcionamiento del algoritmo tendremos que testearlo sobre datos distintos de los usados para entrenar. Por defecto, la librería sklearn ofrece funciones para emplear las métricas más comunes:


<center> <img src="img/matcon.jpg"></center>
<center>Fuente: Confusion Matrix - Applied Deep Learning with Keras</center>


<center> <img src="img/met.png"></center>
<center>Fuente: https://www.researchgate.net/</center>

Primero, apliquemos las metricas a los datos de entrenamiento para observar qué algoritmo ha aprendido mejor, es decir, identificado y extraido más información.

In [14]:
#  Clasificamos los puntos (a través de las features) (Esto puede tardar unos minutos)

# Con el SVM
train_predictions_SVM = clf_svm.predict(train_features)

# Con el Random Forest
train_predictions_RF = clf_rf.predict(train_features)

In [15]:
# Cálculo de métricas para el algoritmo SVM

# Aquí definimos los datos de referencia (ground truth)
y_test = train_labels

# Aquí definimos los datos que hemos calculado
y_pred = train_predictions_SVM

# Calcula y muestra la matriz de confusión
print(confusion_matrix(y_test,y_pred))

# Calcula y muestra estadísticas por clase
print(classification_report(y_test,y_pred))

# Calcula y muestra estadísticas globales
print(accuracy_score(y_test, y_pred))

[[ 1159  5053   826  1350]
 [  134 38379   435  1729]
 [  152  1043 46449  1441]
 [  332  3578  1843  8788]]
              precision    recall  f1-score   support

         1.0       0.65      0.14      0.23      8388
         2.0       0.80      0.94      0.87     40677
         3.0       0.94      0.95      0.94     49085
         4.0       0.66      0.60      0.63     14541

    accuracy                           0.84    112691
   macro avg       0.76      0.66      0.67    112691
weighted avg       0.83      0.84      0.82    112691

0.8410165851753911


# Tarea 4

Analiza los resultados. ¿Cuál es la accuracy? ¿Dónde se produce la mayor confusión? ¿Qué clase se clasifica mejor?

# Tarea 5

Calcula las métricas para el conjunto de entrenamiento con el Random Forest entrenado. ¿El resultado es mejor? 

In [16]:
# Cálculo de métricas para el algoritmo SVM

# Aquí definimos los datos de referencia (ground truth)
y_test = COMPLETAR

# Aquí definimos los datos que hemos calculado
y_pred = COMPLETAR

# Calcula y muestra la matriz de confusión
print(confusion_matrix(y_test,y_pred))

# Calcula y muestra estadísticas por clase
print(classification_report(y_test,y_pred))

# Calcula y muestra estadísticas globales
print(accuracy_score(y_test, y_pred))

[[ 8386     0     2     0]
 [    0 40673     3     1]
 [    2     3 49074     6]
 [    0     4     1 14536]]
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00      8388
         2.0       1.00      1.00      1.00     40677
         3.0       1.00      1.00      1.00     49085
         4.0       1.00      1.00      1.00     14541

    accuracy                           1.00    112691
   macro avg       1.00      1.00      1.00    112691
weighted avg       1.00      1.00      1.00    112691

0.9998047758915973


# Tarea 6

Calcula las métricas para el conjunto de testeo con los dos algoritmos. ¿Cuál fue mejor? ¿Observas underfitting u overfitting en algún algoritmo? ¿Cómo se puede solucionar? **Nota:** Carga la nube de testeo 000079.txt, separa los datos, extrae las features, y clasificalas. Usa las funciones creadas anteriormente.

In [17]:
# Lectura de datos
test_data = COMPLETAR

In [18]:
# Separar datos de entrada
test_coord,test_labels = COMPLETAR

# Extraer características
test_features = COMPLETAR

In [19]:
#  Clasificamos los puntos (a través de las features) (Esto puede tardar unos minutos)

# Con el SVM
test_predictions_SVM = COMPLETAR

# Con el Random Forest
test_predictions_RF = COMPLETAR

In [20]:
# Cálculo de métricas para el algoritmo SVM

# Aquí definimos los datos de referencia (ground truth)
y_test = COMPLETAR

# Aquí definimos los datos que hemos calculado
y_pred = COMPLETAR

# Calcula y muestra la matriz de confusión
print(confusion_matrix(y_test,y_pred))

# Calcula y muestra estadísticas por clase
print(classification_report(y_test,y_pred))

# Calcula y muestra estadísticas globales
print(accuracy_score(y_test, y_pred))

[[ 1920 10956  3556  3270]
 [  221 32952   528  1690]
 [  308   581 39982  1724]
 [  435  4218  1291  7633]]
              precision    recall  f1-score   support

         1.0       0.67      0.10      0.17     19702
         2.0       0.68      0.93      0.78     35391
         3.0       0.88      0.94      0.91     42595
         4.0       0.53      0.56      0.55     13577

    accuracy                           0.74    111265
   macro avg       0.69      0.63      0.60    111265
weighted avg       0.74      0.74      0.69    111265

0.7413562216330383


In [21]:
# Cálculo de métricas para el algoritmo RF

# Aquí definimos los datos de referencia (ground truth)
y_test = COMPLETAR

# Aquí definimos los datos que hemos calculado
y_pred = COMPLETAR

# Calcula y muestra la matriz de confusión
print(confusion_matrix(y_test,y_pred))

# Calcula y muestra estadísticas por clase
print(classification_report(y_test,y_pred))

# Calcula y muestra estadísticas globales
print(accuracy_score(y_test, y_pred))

[[ 5528  5740  4621  3813]
 [  749 32037   355  2250]
 [  840   395 39278  2082]
 [ 1252  2598  1578  8149]]
              precision    recall  f1-score   support

         1.0       0.66      0.28      0.39     19702
         2.0       0.79      0.91      0.84     35391
         3.0       0.86      0.92      0.89     42595
         4.0       0.50      0.60      0.55     13577

    accuracy                           0.76    111265
   macro avg       0.70      0.68      0.67    111265
weighted avg       0.76      0.76      0.74    111265

0.7638700399946075


# Visualización de la clasificación

Una ventaja de las nubes de puntos es que podemos visualizar los resultados, puesto que son datos geométricos. Si usamos la inteligencia artificial para resolver problemas exclusivamente numéricos, no es posible una visualización de los datos tan claro. La visualización de los datos es muy relevante para poder identificar posibles fallos que pasan desapercibidos en las métricas.

Por último, procedemos a exportar los resultados en una nube para poder visualizarla en CloudCompare. La nube exportada tendrá las siguientes columnas: 
- 3 columnas de coordenadas
- 1 columna de labels (ground truth)
- 1 columna de predicción SVM
- 1 columna de predicción RF

¿Se detecta algún problema que no se detectaba en las métricas? ¿Son los resultados tan parecidos como sugieren las métricas? ¿Qué clases se han clasificado mejor? Pudiendo visualizar los problemas, ¿se te ocurre alguna solución mejor que la propuesta solo con las métricas?

In [22]:
# Exportar
# Definicion de la ruta y nombre del archivo
ruta = "Nubes/00000079_predicted.txt"

#Seleccion de datos 
datos = np.column_stack((test_data,test_predictions_SVM,test_predictions_RF))

# Guardado
np.savetxt(ruta,datos,delimiter=' ') 